# Toronto Neighbourhood Clustering

# Submission Part 1

### Make the Necessary imports of libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Make a GET request to obtain the raw HTML text from the Wikipedia page. Convert results into XML format using Beautiful Soup

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_rawHTML = requests.get(wiki_url).text

soup = BeautifulSoup(wiki_rawHTML, 'lxml')

### Retrieve the row values of the main table in the page. Create a dictionary with the table headers as keys and the column values as values

In [3]:
post_table = soup.find('table', {'class':'wikitable sortable'})
post_table_rows = post_table.find_all('tr')

df_dict = {}
dict_keys = []
dict_values = [[],[],[]]

length = len(post_table_rows)
for i in range(length):
    if(i == 0):
        headers = post_table_rows[i].find_all('th')
        for header in headers:
            dict_keys.append(header.text)
    else:
        values = post_table_rows[i].find_all('td')
        for i, value in enumerate(values):
            dict_values[i].append(value.text)
    for i in range(len(dict_keys)):
        df_dict[dict_keys[i]] = dict_values[i]


### Convert the dictionary into a Pandas DataFrame

In [4]:


df = pd.DataFrame.from_dict(df_dict)
df = df[['Postcode', 'Borough', 'Neighbourhood\n']]
df.columns = ['Postcode', 'Borough', 'Neighborhood']
df.head()


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


### Clean the resulting DataFrame, dropping any Not assigned Boroughs

In [5]:
df = df.replace('\n','', regex=True)
df = df.replace('Not assigned', np.nan)
df = df[df['Borough'].isnull() == False]
df.loc[df['Neighborhood'].isnull(), 'Neighborhood'] = df['Borough']
df[df['Borough'] == df['Neighborhood']]
df.reset_index(inplace=True, drop=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Print the Shape of the DataFrame

In [6]:
df.shape

(211, 3)

# Submission Part 2

### Geocoder was not working properly so instead import the csv file into a dataframe

In [7]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the newly created coordinates dataframe with the Neighborhoods dataframe

In [8]:
df_full = pd.merge(df, df_coords, left_on='Postcode', right_on='Postal Code')
df_full.drop(['Postal Code'], axis=1, inplace=True)

df_full.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


# Submission Part 3

### Select Only the Neighborhoods which have 'Toronto' in the Borough name, and drop Postal Code

In [9]:
df_neighborhoods = df_full[df_full['Borough'].str.contains('Toronto')]
df_neighborhoods.drop(['Postcode'], axis=1, inplace=True)
df_neighborhoods.reset_index(drop=True, inplace=True)
df_neighborhoods.head()

C:\Users\allen_xk\Documents\New folder (2)\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Regent Park,43.654260,-79.360636
2,Downtown Toronto,Ryerson,43.657162,-79.378937
3,Downtown Toronto,Garden District,43.657162,-79.378937
4,Downtown Toronto,St. James Town,43.651494,-79.375418


### Define Foursquare API Credentials and constant variables to create the url

In [12]:
CLIENT_ID = 'N0M3U3CG1U2U4TLPUWXXQ101ITC4Y3VRNXWLHYWBP21M0UPC'
CLIENT_SECRET = 'KMEGXCEYQDSJ0OEV25J1O3HUPL1JKAIPR1WJMCQZ2O54CEVT'
VERSION = 20180602

neighborhood_latitude = df_neighborhoods.loc[0, 'Latitude']
neighborhood_longitude = df_neighborhoods.loc[0, 'Longitude']
radius = 500
LIMIT= 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

### Define a function which will return a dataframe containing all venues nearby the neighborhood locations

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    return(nearby_venues)

### Call the function

In [14]:
toronto_venues = getNearbyVenues(df_full['Neighborhood'], df_full['Latitude'], df_full['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design E

### Make a quick examination of the new dataframe

In [15]:
toronto_venues.shape

(4437, 7)

In [16]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


### One-hot encode all of the venue types and create a new dataframe

In [39]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot["Neighborhood"] = toronto_venues["Neighborhood"]
cols_exc_neighborhood = list(toronto_onehot.columns[:-1])
cols_exc_neighborhood.remove('Neighborhood')

fixed_columns = ['Neighborhood'] + cols_exc_neighborhood
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Now we can group rows by neighborhood and take the mean of each category's frequency

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,...,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,Agincourt North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,Albion Gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,Alderwood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0


### Now Get the 10 most common venue types for all Toronto neighborhoods

In [89]:
def return_most_common_venues(row, num_top):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top]
import numpy as np
columns = ['Neighborhood']

for ind in np.arange(10):
    columns.append('#{} most common venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], 10)

neighborhoods_venues_sorted.head()

,Neighborhood,#1 most common venue,#2 most common venue,#3 most common venue,#4 most common venue,#5 most common venue,#6 most common venue,#7 most common venue,#8 most common venue,#9 most common venue,#10 most common venue
0,Adelaide,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Restaurant,Burger Joint,Gym
1,Agincourt,Sandwich Place,Lounge,Breakfast Spot,Clothing Store,Women's Store,Drugstore,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
2,Agincourt North,Playground,Park,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant
3,Albion Gardens,Grocery Store,Pizza Place,Pharmacy,Fast Food Restaurant,Sandwich Place,Coffee Shop,Beer Store,Fried Chicken Joint,Dog Run,Diner
4,Alderwood,Pizza Place,Gym,Skating Rink,Sandwich Place,Athletics & Sports,Pub,Pool,Coffee Shop,Pharmacy,Donut Shop


### Now we may perform k-means clustering

In [90]:
k = 5
clustering_df = toronto_grouped.drop(['Neighborhood'], axis=1)
clustering_df

from sklearn.cluster import KMeans
kmeans_clustering = KMeans(n_clusters=k, random_state=4).fit(clustering_df)
kmeans_clustering.labels_

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 4, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
       2, 0, 2, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 3, 0, 1, 0, 2, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0, 4, 3,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1])

### Create a new dataframe that includes the cluster as well as top 10 venues

In [91]:
neighborhoods_venues_sorted.insert(loc=0, column='Cluster Labels', value=kmeans_clustering.labels_)

In [92]:
toronto_merged = df_full
toronto_merged = toronto_merged.set_index('Neighborhood', drop=True)
neighborhoods_venues_sorted = neighborhoods_venues_sorted.set_index('Neighborhood', drop=True)
neighborhoods_venues_sorted

,Cluster Labels,#1 most common venue,#2 most common venue,#3 most common venue,#4 most common venue,#5 most common venue,#6 most common venue,#7 most common venue,#8 most common venue,#9 most common venue,#10 most common venue
Neighborhood,,,,,,,,,,,
Adelaide,1,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Restaurant,Burger Joint,Gym
Agincourt,1,Sandwich Place,Lounge,Breakfast Spot,Clothing Store,Women's Store,Drugstore,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
Agincourt North,0,Playground,Park,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant
Albion Gardens,1,Grocery Store,Pizza Place,Pharmacy,Fast Food Restaurant,Sandwich Place,Coffee Shop,Beer Store,Fried Chicken Joint,Dog Run,Diner
Alderwood,1,Pizza Place,Gym,Skating Rink,Sandwich Place,Athletics & Sports,Pub,Pool,Coffee Shop,Pharmacy,Donut Shop
Bathurst Manor,1,Coffee Shop,Pharmacy,Sandwich Place,Sushi Restaurant,Bank,Deli / Bodega,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner
Bathurst Quay,1,Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Coffee Shop,Sculpture Garden,Plane,Harbor / Marina,Airport Gate,Airport Food Court
Bayview Village,1,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
Beaumond Heights,1,Grocery Store,Pizza Place,Pharmacy,Fast Food Restaurant,Sandwich Place,Coffee Shop,Beer Store,Fried Chicken Joint,Dog Run,Diner


In [93]:
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted)
toronto_merged.dropna(inplace=True)

In [95]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged

,Postcode,Borough,Latitude,Longitude,Cluster Labels,#1 most common venue,#2 most common venue,#3 most common venue,#4 most common venue,#5 most common venue,#6 most common venue,#7 most common venue,#8 most common venue,#9 most common venue,#10 most common venue
Neighborhood,,,,,,,,,,,,,,,
Adelaide,M5H,Downtown Toronto,43.650571,-79.384568,1,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Restaurant,Burger Joint,Gym
Agincourt,M1S,Scarborough,43.794200,-79.262029,1,Sandwich Place,Lounge,Breakfast Spot,Clothing Store,Women's Store,Drugstore,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
Agincourt North,M1V,Scarborough,43.815252,-79.284577,0,Playground,Park,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant
Albion Gardens,M9V,Etobicoke,43.739416,-79.588437,1,Grocery Store,Pizza Place,Pharmacy,Fast Food Restaurant,Sandwich Place,Coffee Shop,Beer Store,Fried Chicken Joint,Dog Run,Diner
Alderwood,M8W,Etobicoke,43.602414,-79.543484,1,Pizza Place,Gym,Skating Rink,Sandwich Place,Athletics & Sports,Pub,Pool,Coffee Shop,Pharmacy,Donut Shop
Bathurst Manor,M3H,North York,43.754328,-79.442259,1,Coffee Shop,Pharmacy,Sandwich Place,Sushi Restaurant,Bank,Deli / Bodega,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner
Bathurst Quay,M5V,Downtown Toronto,43.628947,-79.394420,1,Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Coffee Shop,Sculpture Garden,Plane,Harbor / Marina,Airport Gate,Airport Food Court
Bayview Village,M2K,North York,43.786947,-79.385975,1,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
Beaumond Heights,M9V,Etobicoke,43.739416,-79.588437,1,Grocery Store,Pizza Place,Pharmacy,Fast Food Restaurant,Sandwich Place,Coffee Shop,Beer Store,Fried Chicken Joint,Dog Run,Diner


### Create a map to visualize the newly created clusters by color

In [96]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [97]:
map_clusters = folium.Map(location=[43.651494, -79.375418], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged.index, toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,#1 most common venue,#2 most common venue,#3 most common venue,#4 most common venue,#5 most common venue,#6 most common venue,#7 most common venue,#8 most common venue,#9 most common venue,#10 most common venue
Neighborhood,,,,,,,,,,,
Agincourt North,Scarborough,Playground,Park,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant
CFB Toronto,North York,Airport,Playground,Park,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
Caledonia-Fairbanks,York,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Greek Restaurant,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant
Downsview,North York,Basketball Court,Bakery,Park,Construction & Landscaping,Women's Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
Downsview East,North York,Airport,Playground,Park,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
Downsview West,North York,Park,Grocery Store,Bank,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
East Toronto,East York,Park,Pizza Place,Coffee Shop,Convenience Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
Forest Hill North,Central Toronto,Trail,Park,Sushi Restaurant,Jewelry Store,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Women's Store
Forest Hill West,Central Toronto,Trail,Park,Sushi Restaurant,Jewelry Store,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Women's Store
